# Narupa Multiplayer Selections Test

This notebook demonstrates how to create a selection via the multiplayer key value store.

In [32]:
from narupa.app.client import NarupaClient
from narupa.mdanalysis import frame_data_to_mdanalysis
from google.protobuf.struct_pb2 import Value, Struct
from google.protobuf.json_format import MessageToDict

## Start a Narupa IMD client

In [33]:
client = NarupaClient()
first_frame = client.wait_until_first_frame(check_interval=0.5, timeout=10)
print(first_frame.particle_count)

5988


In [34]:
client.join_multiplayer("selections test")
multiplayer_client = client._multiplayer_client
player_id = multiplayer_client.player_id
print(player_id)

1


## Create and Modify Selection

In [35]:
selection = client.create_selection("Selection", [0, 1, 2, 3, 4])

### Print the current selection(s)

In [10]:
for selection in client.selections:
    print(selection.selection_name, selection.selected_particle_ids)

Selection 1 {0.0, 1.0, 2.0, 3.0, 4.0}


### Add more indices to the selection

In [ ]:
selection.add_particles([5, 4, 3])
client.update_selection(selection)

### Print the current selection(s)

In [ ]:
for selection in client.selections:
    print(selection.selection_name, selection.selected_particle_ids)

### Replace the selection indices

In [ ]:
selection.set_particles([5, 4, 3])
client.update_selection(selection)

### Print the current selection(s)

In [ ]:
for selection in client.selections:
    print(selection.selection_name, selection.selected_particle_ids)

### Clear the selection

In [29]:
selection.clear_particles()
client.update_selection(selection)

### Print the current selection(s)

In [ ]:
for selection in client.selections:
    print(selection.selection_name, selection.selected_particle_ids)

### Set a selection from MDAnalysis

In [79]:
def generate_mdanalysis_selection(selection: str):
    universe = frame_data_to_mdanalysis(client.first_frame)
    idx_array = universe.select_atoms(selection).indices
    return map(int, idx_array)

selection.set_particles(generate_mdanalysis_selection("bonded name O"))
client.update_selection(selection)

### Set the rendering of a selection

In [77]:
selection.rendering_renderer = {
    'color': 'pink',
    'render': 'ball and stick'
}
client.update_selection(selection)

### Print the current selection(s)

In [59]:
for selection in client.selections:
    print(selection.selection_name, selection.selected_particle_ids)

Selection 1 {1536.0, 1537.0, 1538.0, 1539.0, 1067.0, 1068.0, 1069.0, 1070.0, 1071.0, 1072.0, 1073.0, 1074.0, 1075.0, 1076.0, 1077.0, 569.0, 1082.0, 1081.0, 1084.0, 1085.0, 1086.0, 1087.0, 1088.0, 1089.0, 1090.0, 579.0, 1091.0, 581.0, 582.0, 1092.0, 584.0, 1093.0, 1094.0, 1095.0, 1096.0, 1097.0, 1098.0, 591.0, 592.0, 593.0, 594.0, 595.0, 596.0, 597.0, 598.0, 599.0, 2171.0, 2179.0, 2180.0, 2181.0, 2182.0, 2183.0, 2184.0, 2185.0, 2186.0, 2187.0, 2188.0, 1165.0, 1166.0, 1167.0, 1168.0, 2189.0, 2190.0, 1171.0, 2191.0, 3725.0, 3726.0, 3223.0, 3225.0, 3226.0, 3227.0, 3229.0, 3230.0, 3231.0, 3232.0, 3233.0, 3234.0, 3235.0, 2212.0, 3236.0, 3237.0, 2215.0, 2216.0, 2217.0, 2218.0, 2219.0, 2220.0, 2221.0, 3238.0, 2223.0, 2224.0, 2225.0, 2226.0, 2227.0, 2228.0, 2229.0, 2230.0, 2231.0, 2232.0, 2233.0, 2234.0, 3250.0, 3251.0, 3252.0, 2238.0, 3254.0, 3257.0, 3259.0, 3260.0, 3261.0, 3262.0, 3263.0, 3264.0, 3265.0, 3266.0, 3269.0, 3270.0, 3291.0, 3292.0, 3293.0, 3294.0, 3295.0, 3296.0, 2275.0, 2276.0, 2

# Selection Properties

In [70]:
ligand.set_particles(generate_mdanalysis_selection("resname OSE"))
client.update_selection(ligand)

In [ ]:
ligand = client.create_selection("Ligand", [])
